<figure>
  <img style="float: left;" src="./figures/fair_1.png" width="200"/>
</figure>

# Lab 7 - Fairness of Predictive Models

Today, we will discuss different notions of fairness in the predictive setting.

More specifically, we will go through a running example using the Adult dataset.

Machine learning (ML) usage is increasing every day. Companies and organisations build ML models with the purpose of minimising human effort or improving performance in various tasks. So today Machine learning arguably affects our lives.

Examples:
1. Recommendation system
2. Face recognition
3. Self-driving cars
4. Candidate selection e.x college admissions, CV screening
5. Loan admissions

# Fairness Notions

When machine learning models are being used to make decisions, they cannot be separated from the social and ethical context in which they are applied, and those developing and deploying these models must take care to do so in a manner that accounts for both performance and fairness. 

So in the last decade Fairness has become one of the most active research areas in ML.

The most popular notions are **Individual Fairness** and **Group Fairness**.

### Individual Fairness 
Individual fairness is a notion of fairness, meaning that similar individuals should be treated similarly.

### Group Fairness
In group fairness, different groups should be treated equally, ensuring fair outcomes across demographic groups.

# COMPAS Dataset and Fairness Issues
There are a lot of examples that researchers demonstrate inadvertently discriminating against several population groups.

The most know is [ProPublica's](https://www.propublica.org/datastore/dataset/compas-recidivism-risk-score-data-and-analysis) research in COMPAS Dataset.

The **COMPAS** (Correctional Offender Management Profiling for Alternative Sanctions) tool was used in Florida, to predict the likelihood of defendants reoffending. The model assigns a risk score based on factors like criminal history, age, and demographic data. Judges used these scores to inform decisions about bail, sentencing, and parole.

However, investigations revealed significant **racial bias** in the model’s predictions. 
Black defendants were more likely to be assigned higher risk scores, even when they did not reoffend.
Black defendants had twice "false positive rates" from white defendants.

These disparities led to concerns about **Algorithmic Fairness**.

# A2. Sources of Bias.

Bias may be introduced into a machine learning project at any step along the way, and it is important to carefully think through each potential source and how it may affect your results.


Source of bias:
- Data
    1. Historical injustice.
    2. Sample bias - Collection Bias.
    3. Limited features.
    4. Unbalanced dataset.
    5. Proxy Variables.
- Modelling
    1. Data preprocessing
    2. Model assumptions 
- Feedback loops.
    1. decisions based on biassed models lead to biassed dataset.  

<figure>
  <img style="float: left;" src="./figures/simple_pipeline.png" width="600"/>
</figure>


# A3. Load Dataset

https://archive.ics.uci.edu/ml/datasets/adult

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from collections import OrderedDict

data_types = OrderedDict([
    ("age", "int"),
    ("workclass", "category"),
    ("final_weight", "int"),  # originally it was called fnlwgt
    ("education", "category"),
    ("education_num", "int"),
    ("marital_status", "category"),
    ("occupation", "category"),
    ("relationship", "category"),
    ("race", "category"),
    ("sex", "category"),
    ("capital_gain", "float"),  # required because of NaN values
    ("capital_loss", "int"),
    ("hours_per_week", "int"),
    ("native_country", "category"),
    ("income_class", "category"),
])
target_column = "income_class"

In [ ]:
def read_dataset(path):
    return pd.read_csv(
        path,
        names=data_types,
        index_col=None,

        comment='|',  # test dataset has comment in it
        skipinitialspace=True,  # Skip spaces after delimiter
        na_values={
            'capital_gain': 99999,
            'workclass': '?',
            'native_country': '?',
            'occupation': '?',
        },
        dtype=data_types,
    )

def clean_dataset(data):
    # Test dataset has dot at the end, we remove it in order
    # to unify names between training and test datasets.
    data['income_class'] = data.income_class.str.rstrip('.').astype('category')
    
    # Remove final weight column since there is no use
    # for it during the classification.
    data = data.drop('final_weight', axis=1)
    
    # Duplicates might create biases during the analysis and
    # during prediction stage they might give over-optimistic
    # (or pessimistic) results.
    data = data.drop_duplicates()
    
    # Binary target variable (>50K == 1 and <=50K == 0)
    data[target_column] = (data[target_column] == '>50K').astype(int)
    
    # Categorical dataset
    categorical_features = data.select_dtypes('category').columns
    data[categorical_features] = data.select_dtypes('category').apply(lambda x: x.cat.codes)
    return data

### load & clean dataset

In [ ]:
# get and clean train dataset
TRAIN_DATA_FILE = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
train_data = clean_dataset(read_dataset(TRAIN_DATA_FILE))
train_data = train_data.dropna()
print("Train dataset shape:", train_data.shape)

# get and clean test dataset
TEST_DATA_FILE = 'https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test'
test_data = clean_dataset(read_dataset(TEST_DATA_FILE))
test_data = test_data.dropna()
print("Test dataset shape:", test_data.shape)

In [ ]:
target_column = "income_class"
sensitive_feature = "sex"
features = train_data.columns.difference([target_column])

In [ ]:
train_data.head()

### Data analysis

In [ ]:
plt.figure()
fig = sns.countplot(data=train_data[["income_class","sex"]],
                    x="income_class",
                    hue="sex")
fig.set_xticklabels(['income <= 50K','income > 50K'])
plt.legend(title='sex', labels=['woman', 'men'])
plt.title("Num of values in each category")
plt.show()
plt.close()

### Data Scalling

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = pd.DataFrame(scaler.fit_transform(train_data[features]),columns = features)

X_test = pd.DataFrame(scaler.transform(test_data[features]),columns = features) # note that here we just use the transform method!

The dataset is quite imbalance between a gender, so we expect our model to be unfair.
The source of the bias is coming from our societal bias.

# A4. Train a Model
let's train our model and analyse the results.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=15)

model.fit(X = X_train[features],
          y = train_data[target_column])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
test_data[target_column]

In [ ]:
predictions = model.predict(X_test[features])
accuracy = accuracy_score(y_true=test_data[target_column], y_pred=predictions)

In [ ]:
accuracy

In [ ]:
acc_score = accuracy_score(y_true = test_data[target_column],
                           y_pred = model.predict(X_test[features]))
print("Accuracy_score:", acc_score)

# A5. Analysis of model fairness


Unfortunately, there is no single definition of fairness. Many competing approaches exist to measure whether a model is statistically “fair.” It is essential to start with the social and policy goals for equity and fairness, and then map those goals to the statistical properties we seek in our models to help achieve them. Most definitions involve dividing the population into groups and comparing specific metrics across these groups.

Different definitions prioritize different fairness aspects, so the appropriate choice of fairness definition depends on the application. This typically requires a careful consideration of the project’s goals and a thorough discussion between data scientists, decision-makers, and those impacted by the model's use.

Below, we define some of the most widely recognized fairness definitions:

1. **Demographic Parity** - equal positive outcome rates across groups.
2. **Equalized Opportunities** - equal true positive rates across groups.
3. **Equalized Odds** - equal true positive and false positive rates across groups.
4. **Predictive Parity** - equal positive predictive values (precision) across groups.  
5. **Equal Accuracy** - Equal overall accuracy rates across groups.  
.  
.  

In [ ]:
predictions = model.predict(X_test[features])
predictions = pd.Series(predictions,index = test_data.index)
sensitive_atribute = test_data[sensitive_feature]

In [ ]:
predictions

In [ ]:
woman_mask = sensitive_atribute == 0
predictions_woman = predictions.loc[woman_mask]
labels_woman = test_data.loc[woman_mask, target_column]

acc_score1 = accuracy_score(y_true = labels_woman,
                            y_pred = predictions_woman)
print("Accuracy_score:", acc_score1)

In [ ]:
man_mask = sensitive_atribute == 1
predictions_man = predictions.loc[man_mask]
labels_man = test_data.loc[man_mask, target_column]

acc_score2 = accuracy_score(y_true = labels_man,
                            y_pred = predictions_man)
print("Accuracy_score:", acc_score2)

In [ ]:
Diffrence_on_accuracy = np.abs(acc_score1 - acc_score2)
Diffrence_on_accuracy

## F1.  Demographic Parity

**Demographic parity** , also referred to as **statistical parity** , **acceptance rate parity**  and **benchmarking**

Demographic Parity states that the positive outcome rates between sensitive group must be the same.

---------------
> **Definition Demographic Parity :** We say that a predictor $\hat{Y}$ satisfy **demographic parity** if the predictions $\hat{Y}$ are independent of the sensitive atribute $S$, $\hat{Y} \perp S$.
$$ P_r(\hat{Y}= 1 | S = s ) =  P_r(\hat{Y} = 1), \quad  \forall s\in S $$

in case of 2 sensitive classes, $ s \in \{0,1\}  $ we mesure the bias of model as follows:
$$DP = \; \mid P_r(\hat{Y}= 1 | S = 0 ) -  P_r(\hat{Y}= 1 | S = 1 ) \mid $$  
---------------

In [ ]:
def demographic_parity_difference(predictions, sensitive_attribute):
    """
    Implementation of demographic parity difference for different groups of sensitive attribute
    1. For each group calculate the propotion of positive predictions.
    2. Calcaulte the difference.
    """
    # 1. For each group calculate the proportion of positive predictions.
    # group 0 


    positive_rate_group0 = 

    # group 1 
    
    positive_rate_group1 = 
    
    # 3. Calculate the different.
    difference = np.abs(positive_rate_group0-positive_rate_group1)
    
    return difference, [positive_rate_group0, positive_rate_group1]

In [ ]:
predictions = model.predict(X_test[features])
sensitive_attribute = test_data["sex"]

In [ ]:
demographic_metric, rates = demographic_parity_difference(predictions, sensitive_attribute)

In [ ]:
print(f"Demographic parity difference is: {round(demographic_metric,5)}")

In [ ]:
print(f"The rate of different groups are: {rates}")

---------------
**When to use demographic parity?** 
1. We are aware of **historical biases** that may have affected the quality of our data. E.x not presents of specific minority in a specific work.
2. We want to **change the state of our current world to improve it**. For example, we want to have equal admitions of different group in a specific project.


**Potential issues?** 
1. **Laziness**: We can satisfy demographic parity in we accept random people in group S=0 but qualified people in group  b unless we have the same proportion of positive outcome. We avoid lasyness because classfication is usually perfomred by optimizing an perfomance metric.
2. **Not optimality compatible**: A classifier that satisfy demographic parity is suboptimal, if the dataset demographic parity is not hold.
---------------

## F2. Equalized Opportunities

**Equalized Opportunities** , also referred to as **true positive parity**, **sensitivity**.

Equalized Opportunities states that each group has equal true positive rates.

---------------
> **Definition Equalized Opportunities :** We say that a predictor $\hat{Y}$ satisfy **equalized opportunities** if the predictions $\hat{Y}$ are independent of the sensitive atribute $S$, conditioned on the positive actual outcome \( Y = 1 \), $\hat{Y} \perp S \mid Y = 1$.
$$ P_r(\hat{Y} = 1 | S = s, Y = 1) = P_r(\hat{Y} = 1 | Y = 1), \quad \forall s \in S $$

in case of 2 sensitive classes, $ s \in \{0,1\}  $ we mesure the bias of model as follows:
$$EO = \; \mid P_r(\hat{Y} = 1 | S = 0, Y = 1) -  P_r(\hat{Y} = 1 | S = 1, Y = 1) \mid $$  
---------------

In [ ]:
def equalized_opportunities_difference(predictions, actual, sensitive_attribute):
    """
    Implementation of equalized opportunities difference for different groups of sensitive attribute
    1. For each group calculate the TPR.
    2. Find the maximum and the minim group.
    3. Calculate the difference.
    """
    # 1. For each group calculate the true positive rates.
    # group 0 
   
    true_positive_rate_group0 = 

    # group 1 

    true_positive_rate_group1 = 
    
    # 3. Calculate the different.
    difference = np.abs(true_positive_rate_group0-true_positive_rate_group1)
    
    return difference, [true_positive_rate_group0, true_positive_rate_group1]

In [ ]:
predictions = model.predict(X_test[features])
sensitive_attribute = test_data["sex"]
actual = test_data[target_column]

In [ ]:
from sklearn.metrics import confusion_matrix
equalized_opportunities_metric, _ = equalized_opportunities_difference(predictions, actual, sensitive_attribute)

In [ ]:
print(f"equalized opportunities difference is: {round(equalized_opportunities_metric,5)}")

---------------
**When to use equalized opportunities?**
1. When is a strong emphasis on predicting the positive outcome correctly. e.x college admissions
2. When False Positives are not costly for the stakeholder. e.x spam detection
---------------

## F3. Equalized Odds

**Equalized Odds** , also referred to as

Equalized Odds states that the true positive rates (TPR) and false positive rates (FPR) between sensitive group must be the same.

---------------
> **Definition Equalized Odds :** A classifier $C$ We say that a predictor $\hat{Y}$ satisfy **equalized Odds** if the predictions $\hat{Y}$ are independent of the sensitive atribute $S$, conditioned on the actual outcome \( Y \), $\hat{Y} \perp S \mid Y$.
$$ P_r(\hat{Y} = 1 | S = s, Y = y) = P_r(\hat{Y} = 1 | Y = y), \quad \forall s \in S \quad \forall y \in Y$$

in case of 2 sensitive classes, $ s \in \{0,1\}  $ we want to satisfy both:
$$ P_c(\hat{Y}= 1 | S = 0, Y = 1 ) =  P_c(\hat{Y}= 1 | S = 1, Y = 1 ) $$ 
and 
$$ P_c(\hat{Y}= 1 | S = 0, Y = 0 ) =  P_c(\hat{Y}= 1 | S = 1, Y = 0 ) $$  
---------------
we mesure the bias of model as follows:
$$EOds = \; \mid P_r(\hat{Y} = 1 | S = 0, Y = 1) -  P_r(\hat{Y} = 1 | S = 1, Y = 1) \mid + \mid P_r(\hat{Y} = 1 | S = 0, Y = 0) -  P_r(\hat{Y} = 1 | S = 1, Y = 0) \mid  $$  

In [ ]:
def equalized_odds_difference(predictions, data, sensitive_attribute):
    """
    Implementation of equalized odds difference for different groups of sensitive attribute
    1. For each group calculate the TPR.
    2. Calcaulte the difference.
    3. For each group calculate the TPR.
    4. Calcaulte the difference.
    5. sum the Calcaulte the differences
    """
    # A. For each group calculate the true positive rate.
    # group 0 

    true_positive_rate_group0 = 

    # group 1 
    true_positive_rate_group1 = 
    
    # Calculate the different.
    difference1 = np.abs(true_positive_rate_group0-true_positive_rate_group1)
    
    # B. For each group calculate the false positive rate.
    # group 0 
  
    false_positive_rate_group0 = 
    # group 1 
    
    false_positive_rate_group1 = 
    
    # Calculate the different.
    difference2 = np.abs(false_positive_rate_group0-false_positive_rate_group1)
    
    # Total difference
    difference = 
    return difference, ([true_positive_rate_group0,true_positive_rate_group1], [false_positive_rate_group0,false_positive_rate_group1])

In [ ]:
predictions = model.predict(X_test[features])

In [ ]:
equalized_odds_metrics, _ = equalized_odds_difference(predictions=predictions,
                                                      data=test_data,
                                                      sensitive_attribute=test_data["sex"])

In [ ]:
print(f"equalized odds difference is: {round(equalized_odds_metrics,5)}")

In [ ]:
results = pd.DataFrame([acc_score,
                        demographic_metric,
                        equalized_opportunities_metric,
                        equalized_odds_metrics], 
                       index = ["accuracy",
                                "demographic_metric",
                                "equalized_opportunities",
                                "equalized_odds"],
                       columns = ["model"])

For your project you have explained why you choose the specific fairness criteria.

In [ ]:
def compute_metric(y_pred, data, sensitive_feature):
    # get all metrics
    acc_score = accuracy_score(y_true=data[target_column], y_pred=y_pred)
    
    demographic_metric, _ = demographic_parity_difference(y_pred,
                                                          sensitive_feature)
    
    equalized_opportunities_metric, _ = equalized_opportunities_difference(y_pred,
                                                                           data,
                                                                          sensitive_feature)
    equalized_odds_metrics, _ = equalized_odds_difference(y_pred,
                                                          data,
                                                          sensitive_feature)
    
    return {
            "accuracy": acc_score,
            "demographic_metric": demographic_metric,
            "equalized_opportunities" : equalized_opportunities_metric,
            "equalized_odds": equalized_odds_metrics
            }

# B. Correct the Unfair Classifier

Numerous recent papers have proposed mechanisms to enhance fairness in machine learning algorithms.

In summary, there are three general methods to correct an unfair classifier:

1. **Pre-Processing**: Make changes to the data before training the model, e.g., removing correlated features.
2. **In-Processing**: Make changes to the model to correct fairness, e.g., adding additional loss terms to ensure fairness.
3. **Post-Processing**: Make changes after the model's output, e.g., adjusting classification thresholds.

For an overview of different methods and fairness criteria, we refer you to the following interesting survey: [https://arxiv.org/pdf/2001.09784.pdf](https://arxiv.org/pdf/2001.09784.pdf).

<div>
<img style="float: left;" src="./figures/corect_unfairness.png" width="400"/>
</div>

### 1. Pre-Processing

Unfair models can often result from bias in the data. Pre-processing methods aim to remove bias from the data before it is used to train models. The data is transformed, and then the algorithm is trained as it would be on the original dataset.

Pre-processing techniques work by transforming the data into a different representation to remove bias. Common approaches include re-balancing techniques, removing sensitive features and their proxy variables, or using machine learning techniques, such as representation learning, to transform the data into a fair representation.

**Pros:**
1. Can be applied to any downstream task.
2. Independent of the final classification model.
3. Most approaches do not require access to sensitive features after training.

**Cons:**
1. May reduce performance (e.g., lower accuracy).
2. Cannot address all fairness criteria, as some depend on the classifier. For example, ensuring equalized odds requires access to the classifier's outcome.

### 2. In-Processing

In-Processing techniques incorporate fairness into the optimization process of a classifier. In general, this approach introduces constraints or regularization terms in the objective function during training. Different methods have been proposed, either as general frameworks or solutions based on specific classification algorithms.

**Pros:**
1. Good performance on both accuracy and fairness measures.
2. Flexibility in balancing the trade-off between performance and fairness.
3. No need for access to sensitive features after training.

**Cons:**
1. Certain constraints may be difficult to apply.
2. Limited to specific classifiers.
3. Requires modifying the classifier to include constraints during retraining.

### 3. Post-Processing

These methods post-process the output score of a classifier to remove bias. The most common techniques use different classification thresholds for each group to satisfy fairness criteria.

**Pros:**
1. Can be applied after any classifier.
2. Relatively good performance, especially on fairness measures.
3. No need to modify the classifier itself.

**Cons:**
1. Often requires test-time access to the protected attribute.

# B. Pre-Processing

### B1 Unawareness
A very intuitive but **wrong solution** is to assume that ignoring the sensitive feature we can build a fair classifier.
This approach is wrong because other feature may be correlated with our sensitive features.

We will build a **Unawareness** classifier and the compare the results with our previous classifier.

## Compare the results of 2 models

In [ ]:
all_results = pd.concat([results, unaware_results],axis=1)
all_results

In [ ]:
all_results.plot(kind="bar")

### Conclusion
Our conclusion is that we are not able to vanish unfairness by excluding sensitive features.  
The reason in that some other feature leak information about our sensitive attribute.

## B2 Remove correlated features
From the above results we expect that our sensitive feature are correlate with non-sensitive feature in our analysis.
We will try to find and exclude also those feature in order to improve our fairness properties.

In [ ]:
corr = {}
for f in non_sensitive_features:
    corr[f] =np.corrcoef(test_data[sensitive_feature], test_data[f])[0,1]
corr = np.abs(pd.DataFrame(data=corr, index=["corr"]).T).sort_values(by="corr",ascending = False)

In [ ]:
plt.figure()
corr.plot(kind="bar")
plt.title("Correlation of sensitive attribute")
plt.show()
plt.close()

#### Iteratively Exclude Correlated Features

Let's build a classifier while progressively removing the most correlated features in each step. This helps us see how removing correlated features affects model accuracy.

### Steps

1. **Get Feature Correlations**  
   Calculate the correlation matrix and identify the most correlated features.

2. **Iterate Over Top Correlated Features**  
   For each step \( k \):
   - Exclude the top \( k \) most correlated features.
   - Train a classifier with the remaining features.
   - Measure fairness

In [ ]:
k=3
k_most_correlated = corr.iloc[0:k].index.values.tolist()

In [ ]:
for i in range(len(k_most_correlated)):
    print(f"===========iteration{i+1}===========")
    
    # 1. train
   
    # 2. compute matrix
   
    # 3. append result

In [ ]:
all_results

In [ ]:
all_results.plot(kind="bar")

# Things to Remember

1. **ML models can be biased due to various sources,** with data-dependent biases being the most common.
2. **Selecting the appropriate fairness criterion** is essential and should align with the specific requirements of our problem.
3. **Fairness metrics can be calculated** using different rates for different groups.
4. **Removing sensitive features does not guarantee fairness** because proxy variables can still introduce bias. In many cases, removing these features does not significantly influence the algorithm.
5. **Biases can be addressed through pre-processing, in-processing, or post-processing methods.**
6. **Different methods involve trade-offs** between fairness and model performance, so choosing the right approach depends on the acceptable balance between these two factors.

For an overview of different methods and fairness critiriaa we refer you to the following survey https://arxiv.org/pdf/2001.09784.pdf.

Fairness python libraries:
https://aif360.readthedocs.io/en/stable/modules/algorithms.html
https://fairlearn.org/